In [ ]:
!git clone https://github.com/patrickfrank1/chess-embedding-experiments.git

Upload `.env` file into project directory `/content/chess-embedding-experiments/colab`.

In [ ]:
# This script makes stateful changes and should be sourced, it will
# 1. clone the github repo if it does not yet exist
# 2. install the dependencies
# 3. download the data sets
# 4. set the project path to /content/chess-embedding-experiments
!source /content/chess-embedding-experiments/colab/setup.sh

In [ ]:
%cd /content/chess-embedding-experiments

In [ ]:
!python data.py

In [ ]:
!python train.py

Quick evaluation

In [ ]:
path = !(echo $(pwd))

In [ ]:
import sys
sys.path.insert(0, path[0])

import numpy as np
import chess
import tensorflow
from tensorflow import keras

from utils.board_representation import tensor_to_board
from utils.visualisation import binarize_array, print_board
from utils.data_loader import load_train_test
from train import sum_squared_loss, pc_column_reg, pc_plane_reg, num_pc_reg, custom_regularized_loss

In [ ]:
train_data, test_data = load_train_test("./data", "position_tensor_000")
print(len(train_data), len(test_data))

In [ ]:
model = keras.models.load_model(
    "./model/20230915_trivial_autoencoder.keras",
    custom_objects={
        "sum_squared_loss": sum_squared_loss,
        "num_pc_reg": num_pc_reg,
        "pc_column_reg": pc_column_reg,
        "pc_plane_reg": pc_plane_reg,
        "custom_regularized_loss": custom_regularized_loss
    }
)

In [ ]:
indices = [2]
for index in indices:
    print(f"Compare boards for index {index}")
    sample = test_data[index].reshape((1,8,8,15))
    reconstruction = model.predict_on_batch(sample)
    print(model.evaluate(x=[reconstruction], y=[sample]))
    print(custom_regularized_loss(sample, reconstruction), sum_squared_loss(sample, reconstruction), num_pc_reg(sample, reconstruction), pc_column_reg(sample, reconstruction), pc_plane_reg(sample, reconstruction))
    binary_reconstruction = binarize_array(reconstruction)
    sample_board = tensor_to_board(sample[0])
    reconstructed_board = tensor_to_board(binary_reconstruction[0])
    print_board(sample_board, "\n")
    print_board(reconstructed_board)

If that's promising then try to save artifacts

In [ ]:
!dvc add model

In [ ]:
!git add --all && git status

In [ ]:
!git commit -m "Colab model loss xy at batch yz"

In [ ]:
!git push origin master

In [ ]:
!dvc push -r colab_origin